In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

%matplotlib inline

In [4]:
data_01 = pd.read_csv(f'mba_decision_dataset.csv')
data_v2 = data_01.copy()

data_v2.head()

,Person ID,Age,Gender,Undergraduate Major,Undergraduate GPA,Years of Work Experience,Current Job Title,Annual Salary (Before MBA),Has Management Experience,GRE/GMAT Score,Undergrad University Ranking,Entrepreneurial Interest,Networking Importance,MBA Funding Source,Desired Post-MBA Role,Expected Post-MBA Salary,Location Preference (Post-MBA),Reason for MBA,Online vs. On-Campus MBA,Decided to Pursue MBA?
0,1,27,Male,Arts,3.18,8,Entrepreneur,90624,No,688,185,7.9,7.6,Loan,Finance Manager,156165,International,Entrepreneurship,On-Campus,Yes
1,2,24,Male,Arts,3.03,4,Analyst,53576,Yes,791,405,3.8,4.1,Loan,Startup Founder,165612,International,Career Growth,Online,No
2,3,33,Female,Business,3.66,9,Engineer,79796,No,430,107,6.7,5.5,Scholarship,Consultant,122248,Domestic,Skill Enhancement,Online,No
3,4,31,Male,Engineering,2.46,1,Manager,105956,No,356,257,1.0,5.3,Loan,Consultant,123797,International,Entrepreneurship,On-Campus,No
4,5,28,Female,Business,2.75,9,Entrepreneur,96132,No,472,338,9.5,4.9,Loan,Consultant,197509,Domestic,Skill Enhancement,Online,Yes


In [12]:
data_v2.columns

Index(['Person ID', 'Age', 'Gender', 'Undergraduate Major',
       'Undergraduate GPA', 'Years of Work Experience', 'Current Job Title',
       'Annual Salary (Before MBA)', 'Has Management Experience',
       'GRE/GMAT Score', 'Undergrad University Ranking',
       'Entrepreneurial Interest', 'Networking Importance',
       'MBA Funding Source', 'Desired Post-MBA Role',
       'Expected Post-MBA Salary', 'Location Preference (Post-MBA)',
       'Reason for MBA', 'Online vs. On-Campus MBA', 'Decided to Pursue MBA?'],
      dtype='object')

In [28]:
# Pipeline for the feature droping 

def drop_column(X, column_name = 'Person ID'):
    return X.drop(columns = [column_name], errors = 'Ignore')

drop_column_transformer = FunctionTransformer(drop_column, kw_args = {'column_name': 'Person ID'})

Drop_Feature_Pipeline = Pipeline([
    ('drop_column', drop_column_transformer)
])
Drop_Feature_Pipeline

Pipeline(steps=[('drop_column',
                 FunctionTransformer(func=<function drop_column at 0x108f70e00>,
                                     kw_args={'column_name': 'Person ID'}))])

In [41]:
# Pipeline for the feature scaling 

numerical_data = data_v2.select_dtypes(include = ['float64', 'int64']).columns

Scaling_Pipeline = Pipeline([
    ('preprocessor', ColumnTransformer([
    ('scaler', StandardScaler(), numerical_data)
    ]))
])

Scaling_Pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  Index(['Person ID', 'Age', 'Undergraduate GPA', 'Years of Work Experience',
       'Annual Salary (Before MBA)', 'GRE/GMAT Score',
       'Undergrad University Ranking', 'Entrepreneurial Interest',
       'Networking Importance', 'Expected Post-MBA Salary'],
      dtype='object'))]))])

In [47]:
# Pipeline for the feature encoding 

data1 = data_v2.select_dtypes(include = ['object'])

# Sample DataFrame (Replace with actual dataset)
data_v2 = pd.DataFrame({}) # Replace with your actual dataset

# Columns for Encoding
label_cols = ['Has Management Experience', 'Location Preference (Post-MBA)', 'Online vs. On-Campus MBA', 'Decided to Pursue MBA?']
onehot_cols = ['Gender', 'Undergraduate Major', 'Curren Job Title', 'MBA Funding Source', 'Desired Post-MBA Role', 'Reason for MBA']

# Label Encoding Transformation Function
def label_encode_columns(X):
    X = X.copy()
    le = LabelEncoder()
    for col in label_cols:
        X[col] = le.fit_transform(X[col])
    return X

label_encoder_transformer = FunctionTransformer(label_encode_columns, validate = False)


# Create ColumnTransformer with Feature Names Handling
preprocessor = ColumnTransformer([
    ('OneHotEncoder', OneHotEncoder(sparse_output = False, drop = 'first'), onehot_cols),
    ('LabelEncoder', FunctionTransformer(label_encode_columns, validate = False), label_cols)
], remainder = 'passthrough')

# Define a function to convert output into a Dataframe 
def convert_to_dataframe(X, pipeline):
    feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
    return pd.DataFrame(X, columns = feature_names)

# Create a Pipeline
Encoding_Pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('convert_data', FunctionTransformer(lambda X: convert_to_dataframe(X, pipeline), validate = False))
])

Encoding_Pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('OneHotEncoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Gender',
                                                   'Undergraduate Major',
                                                   'Curren Job Title',
                                                   'MBA Funding Source',
                                                   'Desired Post-MBA Role',
                                                   'Reason for MBA']),
                                                 ('LabelEncoder',
                                                  FunctionTransformer(func=<function label_encode_columns at 0x1094a3740>),
                                                  ['Has Management Experience',
                                                   'Location Preference '
                                                   '(Post-MBA)',
                                                   'Online vs. On-Campus MBA',
                                                   'Decided to Pursue '
                                                   'MBA?'])])),
                ('convert_data',
                 FunctionTransformer(func=<function <lambda> at 0x1094a3380>))])